<a href="https://colab.research.google.com/github/rafasacaan/learn-something/blob/main/learn_sqlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# learn-sqlite3

Pros:
- everything

Cons:
  - not good a concurrent write data

Let´s imagine we want to build a small employee database. Let´s connect to DB if exists, else, create the DB.

In [1]:
import sqlite3

conn = sqlite3.connect("employee.db")

An empty file **employee.db** for the database is created.

In [27]:
!ls -lh employee.db

-rw-r--r-- 1 root root 8.0K Dec  2 14:56 employee.db


Another way is to not create a database, and store the data in RAM.

In [ ]:
import sqlite3

conn = sqlite3.connect(":memory:")

Now, lets create a **cursor**, which lets us run SQL commands.

In [3]:
c = conn.cursor()

We can now run commands. For example, we can **create a table**. The three commands we will use are:
1. execute
2. commit
3. close connection

In [4]:
c.execute("""
  CREATE TABLE employees (
  first text,
  last text,
  pay integer
)""")

conn.commit()
conn.close()

Lets insert a sample of data

In [5]:
conn = sqlite3.connect("employee.db")
c = conn.cursor()

In [6]:
c.execute("""
  INSERT INTO employees VALUES (
  'Juan', 'Salvador', 10000
)""")

c.execute("""
  INSERT INTO employees VALUES (
  'Pedro', 'Salvador', 5000
)""")

c.execute("""
  INSERT INTO employees VALUES (
  'Pablo', 'Santos', 1000
)""")

conn.commit()
conn.close()

Lets now query our table

In [10]:
conn = sqlite3.connect("employee.db")
c = conn.cursor()

In [18]:
c.execute("""
  SELECT * FROM employees WHERE last = 'Salvador'
""")

c.fetchone() # fetch next row

('Juan', 'Salvador', 10000)

In [19]:
c.execute("""
  SELECT * FROM employees WHERE last = 'Salvador'
""")

c.fetchmany(3) # ask for 3 results

[('Juan', 'Salvador', 10000), ('Pedro', 'Salvador', 5000)]

In [20]:
c.execute("""
  SELECT * FROM employees WHERE last = 'Salvador'
""")

c.fetchall()

[('Juan', 'Salvador', 10000), ('Pedro', 'Salvador', 5000)]

Now lets create an employee class in python

In [21]:
class Employee:
  """A sample employee class"""

  def __init__(self, first, last, pay):
    self.first = first
    self.last = last
    self.pay = pay

  @property
  def email(self):
    return "{}.{}@email.com".format(self.first, self.last)

  @property
  def fullname(self):
    return "{} {}".format(self.first, self.last)

  @property
  def __repr__(self):
    return "Employee('{}', '{}', {})".format(self.first, self.last, self.pay)

To ensure proper data injection security-wise, we can do the following.

In [24]:
#from file_name import Employee

emp_1 = Employee('John','Doe', 1000)
emp_2 = Employee('Jane','Doe', 3000)

c.execute("INSERT INTO employees VALUES (:first, :last, :pay)", {'first': emp_1.first, 'last': emp_1.last, 'pay': emp_1.pay})
c.execute("INSERT INTO employees VALUES (:first, :last, :pay)", {'first': emp_2.first, 'last': emp_2.last, 'pay': emp_2.pay})
conn.commit()

In [28]:
c.execute("""SELECT * FROM employees WHERE last = 'Doe'""")
c.fetchall()

[('John', 'Doe', 1000), ('John', 'Doe', 1000), ('Jane', 'Doe', 3000)]

Create functions to handle custom database operations.

In [ ]:
def update_pay(emp, pay):
    with conn:
        c.execute("""UPDATE employees SET pay = :pay
                    WHERE first = :first AND last = :last""",
                  {'first': emp.first, 'last': emp.last, 'pay': pay})


def remove_emp(emp):
    with conn:
        c.execute("DELETE from employees WHERE first = :first AND last = :last",
                  {'first': emp.first, 'last': emp.last})